In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os
from torch.utils.data import Dataset
from nn import CustomImageDataset, FeedForwardNN, validate, CNN, train_model, set_seed, CNN_LSTM, CNNTransformer
from testing_models import test_model
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [2]:
input_size = 28 * 28  # Flattened image size (28x28)
num_classes = 94  # Number of classes (modify if needed)
learning_rate = 0.001
num_epochs = 20
batch_size = 64
set_seed(42)

dataset = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="train")
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataset_test = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="test")
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
val_dataset = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="validation")
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)




In [ ]:
# Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(input_size=input_size, num_classes=num_classes, hidden_size=288).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0

for epoch in range(num_epochs):
    # Training step
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(data_loader):.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print("Validation loss improved! Saving model...")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break
    scheduler.step(val_loss)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(input_size=input_size, num_classes=num_classes, hidden_size=288).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0
train_model(num_epochs, data_loader, val_loader, device, model, criterion, optimizer, scheduler, patience)

In [ ]:
torch.save(model, "FeedForward.pth")

In [ ]:
dataset_test = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="test")
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
test_model(model, data_loader_test, device, criterion)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0

for epoch in range(num_epochs):
    # Training step
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(data_loader):.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print("Validation loss improved! Saving model...")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break
    scheduler.step(val_loss)

Epoch [1/20], Train Loss: 2.2303, Validation Loss: 1.2888, Validation Accuracy: 63.99%
Validation loss improved! Saving model...


KeyboardInterrupt: 

In [ ]:
test_model(model, data_loader_test, device, criterion)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_LSTM(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0
train_model(num_epochs, data_loader, val_loader, device, model, criterion, optimizer, scheduler, patience)

In [ ]:
test_model(model, data_loader_test, device, criterion)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNTransformer(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0
#train_model(num_epochs, data_loader, val_loader, device, model, criterion, optimizer, scheduler, patience)

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
test_model(model, data_loader_test, device, criterion)

In [ ]:
torch.save(model,"transformer.pth")

In [4]:
test = torch.load("transformer.pth")
test_model(test,data_loader_test, device, criterion)

/var/folders/92/3_42v5w111v7m641kr368p5c0000gn/T/ipykernel_36113/2581909732.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test = torch.load("transformer.pth")


Model: <class 'nn.CNNTransformer'>
Accuracy: 0.8664
Precision: 0.8689
Recall: 0.8664
F1 Score: 0.8640
Confusion Matrix:
[[  40    0    0 ...    0    0    0]
 [   0   45    0 ...    0    0    0]
 [   0    0   19 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   1    0    0 ...    0    9    0]
 [   0    0    0 ...    0    0 1010]]




/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.391109123322017,
 86.64303144712004,
 0.8639561695556224,
 array([[  40,    0,    0, ...,    0,    0,    0],
        [   0,   45,    0, ...,    0,    0,    0],
        [   0,    0,   19, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   1,    0,    0, ...,    0,    9,    0],
        [   0,    0,    0, ...,    0,    0, 1010]]))